In [1]:
from langgraph.checkpoint.memory import InMemorySaver

from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model


llm = init_chat_model("anthropic:claude-3-5-sonnet-latest")
memory = InMemorySaver()


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)


In [2]:
graph = graph_builder.compile(checkpointer=memory)

In [3]:
config = {"configurable": {"thread_id": "1"}}

user_input = "Hi there! My name is Pierre."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there! My name is Pierre.
================================== Ai Message ==================================

Hi Pierre! Nice to meet you. I'm Claude. How can I help you today?


In [4]:
user_input = "Remember my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember my name?
================================== Ai Message ==================================

Yes, you told me your name is Pierre.


In [5]:
# The only difference is we change the `thread_id` here to "2" instead of "1"
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember my name?
================================== Ai Message ==================================

I actually don't remember your name from any previous conversations. Each conversation with me starts fresh - I don't maintain memory of past chats. If you'd like me to know your name, feel free to tell me!


In [6]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='Hi there! My name is Pierre.', additional_kwargs={}, response_metadata={}, id='972bea06-56b0-432f-a34a-3ddee9637078'), AIMessage(content="Hi Pierre! Nice to meet you. I'm Claude. How can I help you today?", additional_kwargs={}, response_metadata={'id': 'msg_01USS9bRFwJraWUWHq5HQ7Gt', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 15, 'output_tokens': 22, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run--0a8b752b-062e-4e0b-8daf-67f776df6dc2-0', usage_metadata={'input_tokens': 15, 'output_tokens': 22, 'total_tokens': 37, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}), HumanMessage(content='Remember my name?', additional_kwargs={}, response_metadata={}, id='b2a3dedb-d940-42db-8dfa-abc8601d6fb9'), AIMessage(content='Yes, you told